# notebook for further prototyping:

why I need to include intent='inout' for state, and have to add init process:

In [4]:
import numpy as np
import xsimlab as xs

In [65]:
# first build 1.
@xs.process
class OneState:
    
    state = xs.variable(intent='inout')
    
    fluxes = xs.group('flux')
    
    def initialize(self):
        self.state=1
    
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step

    def finalize_step(self):
        self.state += self.delta

@xs.process
class OtherState(OneState):
    state = xs.variable(intent='out')
    
    
@xs.process
class OneFlux:
    
    flux1 = xs.variable(intent='out', groups='flux')
    
    state1 = xs.foreign(OneState,'state')
    
    def run_step(self):
        self.flux1 = 0.1 * self.state1

In [66]:
# This is necessary when setting state intent 'inout', and reusing it in the Flux calculation as xs.foreign

@xs.process
class Init:
    
    state = xs.foreign(OneState,'state', intent='out')
    
    def initialize(self):
        self.state = 0.1

## this works:

In [67]:
MinimalStateFluxModel = xs.Model({'State':OneState, 'Flux':OneFlux, 'Setup':Init})

### this gives a cycle dependency error:

In [69]:
out_MinimalStateFluxModel = xs.Model({'State':OtherState, 'Flux':OneFlux})

RuntimeError: Cycle detected in process graph: State->Flux->State

In [61]:
in_ds = xs.create_setup(
    model=MinimalStateFluxModel,
    clocks={   
        'time': np.linspace(0,2,30),
    },
    master_clock='time',
    #input_vars={'State__state':0.2},
    output_vars={'State__state':'time'}
                 )

In [62]:
with MinimalStateFluxModel:
    out_ds = in_ds.xsimlab.run()

In [63]:
out_ds

<xarray.Dataset>
Dimensions:       (time: 30)
Coordinates:
  * time          (time) float64 0.0 0.06897 0.1379 0.2069 ... 1.862 1.931 2.0
Data variables:
    State__state  (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1

# further prototyping:

Questions:
1. how to best handle the process class constructor, for my purposes 
(i.e. to be able to flexibily initialize base components in different dimensions)

# below here, test process class factory:

i.e. Question 1

IDEA: I can use test-case dimensions!

PhysEnv collects label and state of each component, via xs.group!()

and distributes fluxes from there! it collects forcing fluxes, and passes them down (thanks to xs.group() labels.

as base component, something like this:
@xs.process
class Component:
    """Base class for a component of a ecosystem."""

    label = xs.variable(groups='c_labels', description='component label')
    
    c0 = xs.variable(
        # only slab model (scalar)
        dims=(), 
        # support slab + 2D models ?
        #dims=[(), ('lat', 'lon')],
        groups='c_c0',
        description='inital concentration'
    )

In [ ]:
@xs.process
class AllComponents:
    """Group some component propreties as 1-d arrays."""
    
    c_labels = xs.group('c_labels')
    c_c0 = xs.group('c_c0')
    
    labels = xs.variable(
        dims='component',
        intent='out',
        description='component labels'
    )
    c0 = xs.variable(
        dims='component',
        intent='out',
        description='initial concentration'
    )
    
    # use component labels as xarray coordinate/index
    component = xs.index(
        dims='component',
        description='component label'
    )
    
    def initialize(self):
        self.labels = np.array(list(self.c_labels))
        self.component = self.labels
        self.c0 = np.array(list(self.c_c0), dtype=np.double)